# FFT Implementation for Feature Extraction
**Author:** [Divyanshu Raghuwanshi](https://www.linkedin.com/in/divyanshu-raghuwanshi-85037b160/)<br>
**Date created:** 2020/07/02<br>

In [1]:
import numpy as np
from scipy import fftpack
import pandas as pd
import matplotlib.pyplot as plt
import statistics

import xlsxwriter
import glob
import os

In [2]:
freqs=[[12.288, 24.7808, 37.888, 50.176, 54.4768], [12.288,24.9856,37.0688,50.176,54.272],
       [12.288,24.3712,37.2736,50.5856,53.8624], [12.288,25.6,37.6832,50.3808,54.4768]]

In [5]:
outWorkbook=[xlsxwriter.Workbook("750rpm.xlsx"), xlsxwriter.Workbook("1500rpm.xlsx"), xlsxwriter.Workbook("2250rpm.xlsx"), 
            xlsxwriter.Workbook("3000rpm.xlsx"), xlsxwriter.Workbook("3250rpm.xlsx")]

df=[[pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/0.5mm/12.288.csv'),
    pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/0.5mm/24.7808.csv'),
    pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/0.5mm/37.888.csv'),
    pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/0.5mm/50.176.csv'),
    pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/0.5mm/54.4768.csv')],
    
     [pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.0mm/12.288.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.0mm/24.9856.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.0mm/37.0688.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.0mm/50.176.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.0mm/54.272.csv')],
   
    [pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.5mm/12.288.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.5mm/24.3712.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.5mm/37.2736.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.5mm/50.5856.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/1.5mm/53.8624.csv')],
    
    [pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/2.0mm/12.288.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/2.0mm/25.6.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/2.0mm/37.6832.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/2.0mm/50.3808.csv'),
     pd.read_csv('D:/kit.me/external_data/horizontal-misalignment/horizontal-misalignment/2.0mm/54.4768.csv')],
   ]

#x=0

for f in range(5):
    outSheet= outWorkbook[f].add_worksheet()
    outSheet.write("A1","Time")

    outSheet.write("B1","axial1")
    outSheet.write("C1","radial1")

    outSheet.write("D1","axial2")
    outSheet.write("E1","radial2") 

    outSheet.write("F1","amp1x-R1")
    outSheet.write("G1","amp2x-R1")
    outSheet.write("H1","amp3x-R1")
    outSheet.write("I1","phase1x-R1")
    outSheet.write("J1","phase2x-R1")
    outSheet.write("K1","phase3x-R1")

    
    outSheet.write("L1","amp1x-R2")
    outSheet.write("M1","amp2x-R2")
    outSheet.write("N1","amp3x-R2")
    outSheet.write("O1","phase1x-R2")
    outSheet.write("P1","phase2x-R2")
    outSheet.write("Q1","phase3x-R2")
    
    x=0
    
    for load in range(4):

        df[load][f] = df[load][f].iloc[:,:]
        df[load][f] = df[load][f].values

        s_w=10 #sliding window
        l_a_o =5000 #length taken at once

        for i in range(4):
            if(i<2):
                sig = df[load][f][:,i+1:i+2] #(i+2)th column
            if(i>1):
                sig = df[load][f][:,i+2:i+3]

            for j in range(24500):
                rat=sig[(s_w*j):((s_w*j)+l_a_o), :]
                value= rat.sum(axis = 0)

                value/=l_a_o
                outSheet.write(x+j+1, i+1, value)

        #x+= ((250000-l_a_o)//s_w)+1


        for i in range(5):
                if(i==1 or i==4):
                    sig = df[load][f][:,i+1:i+2] #(i+2)th column
                else:
                    continue
                for j in range(((250000-l_a_o)//s_w)):
                    rat=sig[(s_w*j):((s_w*j)+l_a_o), :]

                    rat_length =  l_a_o #this is 5 x 1000, length is changable
                    rot_freq = freqs[load][f]
                    rat = np.round(rat,2)

                    freq= 1024
                    time_step = 1/freq
                    time_vect = np.arange(0,5,time_step) # 5 is the number of cycles

                    rat_fft = fftpack.fft(rat)



                    amplitude = np.abs(rat_fft)
                    power = amplitude**2
                    angle = np.angle(rat_fft)

                    sampling_freq = fftpack.fftfreq(rat_length, time_step)
                    sampling_freq = np.reshape(sampling_freq,(rat_length,1))

                    amp_freq = np.array([amplitude,sampling_freq])
                    angl_freq = np.array([angle,sampling_freq])

                    amp_freq = amp_freq.reshape(amp_freq.shape[0], amp_freq.shape[1])
                    Freq_res = sampling_freq[2,:]-sampling_freq[1,:]
                    Freq_resolution = Freq_res.item()

                    pos1x = int(round(1*rot_freq ,0))
                    pos2x = int(round(2*rot_freq ,0))
                    pos3x = int(round(3*rot_freq ,0))

                    amp1x = amp_freq[0,pos1x]
                    phase1x = angl_freq[0,pos1x]

                    amp2x = amp_freq[0,pos2x]
                    phase2x = angl_freq[0,pos2x]

                    amp3x = amp_freq[0,pos3x]
                    phase3x = angl_freq[0,pos3x]

                    #print('amp1x =' + str(amp1x) + '\nphase1x=' + str(phase1x)+'\namp2x =' + str(amp2x) + '\nphase2x=' + str(phase3x)+'\namp3x =' + str(amp3x) + '\nphase3x=' + str(phase1x))
                    if(i==1):
                        outSheet.write(x+j+1, i+4, amp1x) #5th col
                        outSheet.write(x+j+1, i+5, amp2x)
                        outSheet.write(x+j+1, i+6, amp3x)
                        outSheet.write(x+j+1, i+7, phase1x)
                        outSheet.write(x+j+1, i+8, phase2x)
                        outSheet.write(x+j+1, i+9, phase3x)

                    if(i==4):
                        outSheet.write(x+j+1, i+7, amp1x) #11th col
                        outSheet.write(x+j+1, i+8, amp2x)
                        outSheet.write(x+j+1, i+9, amp3x)
                        outSheet.write(x+j+1, i+10, phase1x)
                        outSheet.write(x+j+1, i+11, phase2x)
                        outSheet.write(x+j+1, i+12, phase3x)

        x+= ((250000-l_a_o)//s_w)

    outWorkbook[f].close()